# Data Merging 

## Reading in Datasets

* Read the datasets in and add a merge column, which is comprised of date, county, and state. Headers printed to ensure correct loading and modification of data. Columns were renamed in excel to a python-friendly name (i.e. _ instead of spaces) 

In [1]:
import pandas as pd
import csv

In [2]:

AQData = pd.read_csv('AQData.csv')
AQData = AQData.drop(['Unnamed: 0'], axis=1)
covidData = pd.read_csv('cleanedCovidData.csv')
covidData = covidData.drop(['Unnamed: 0'], axis=1)

In [3]:
AQData.head()

,Date_Local,State_Name,County_Name,Arithmetic_Mean_ppb,AQI_ozone,Arithmetic_Mean_cleaned,AQI_no2,Arithmetic_Mean,AQI,uniqueID
0,5/1/2020,Alabama,Baldwin,35.000000,50.000000,NaN,NaN,NaN,NaN,"Baldwin, Alabama"
1,5/1/2020,Alabama,DeKalb,47.000000,58.000000,NaN,NaN,NaN,NaN,"DeKalb, Alabama"
2,5/1/2020,Alabama,Elmore,28.000000,44.000000,NaN,NaN,NaN,NaN,"Elmore, Alabama"
3,5/1/2020,Alabama,Etowah,34.000000,51.000000,NaN,NaN,NaN,NaN,"Etowah, Alabama"
4,5/1/2020,Alabama,Jefferson,30.333333,47.666667,15.311553,32.0,11.0125,46.0,"Jefferson, Alabama"


In [4]:
covidData.head()

,date,county,state,fips,cases,deaths,uniqueId,daily cases,daily deaths
0,5/1/2020,Snohomish,Washington,53061,2466,108,"Snohomish, Washington",20,0
1,5/2/2020,Snohomish,Washington,53061,2492,108,"Snohomish, Washington",26,0
2,5/3/2020,Snohomish,Washington,53061,2737,108,"Snohomish, Washington",245,0
3,5/4/2020,Snohomish,Washington,53061,2784,110,"Snohomish, Washington",47,2
4,5/5/2020,Snohomish,Washington,53061,2807,110,"Snohomish, Washington",23,0


In [5]:
AQData['mergeID'] = AQData['County_Name'] + ', ' + AQData['State_Name'] + ', ' + AQData['Date_Local']
covidData['mergeID'] = covidData['county'] + ', ' + covidData['state'] + ', ' + covidData['date']

In [6]:
AQData.head()

,Date_Local,State_Name,County_Name,Arithmetic_Mean_ppb,AQI_ozone,Arithmetic_Mean_cleaned,AQI_no2,Arithmetic_Mean,AQI,uniqueID,mergeID
0,5/1/2020,Alabama,Baldwin,35.000000,50.000000,NaN,NaN,NaN,NaN,"Baldwin, Alabama","Baldwin, Alabama, 5/1/2020"
1,5/1/2020,Alabama,DeKalb,47.000000,58.000000,NaN,NaN,NaN,NaN,"DeKalb, Alabama","DeKalb, Alabama, 5/1/2020"
2,5/1/2020,Alabama,Elmore,28.000000,44.000000,NaN,NaN,NaN,NaN,"Elmore, Alabama","Elmore, Alabama, 5/1/2020"
3,5/1/2020,Alabama,Etowah,34.000000,51.000000,NaN,NaN,NaN,NaN,"Etowah, Alabama","Etowah, Alabama, 5/1/2020"
4,5/1/2020,Alabama,Jefferson,30.333333,47.666667,15.311553,32.0,11.0125,46.0,"Jefferson, Alabama","Jefferson, Alabama, 5/1/2020"


In [7]:
covidData.head()

,date,county,state,fips,cases,deaths,uniqueId,daily cases,daily deaths,mergeID
0,5/1/2020,Snohomish,Washington,53061,2466,108,"Snohomish, Washington",20,0,"Snohomish, Washington, 5/1/2020"
1,5/2/2020,Snohomish,Washington,53061,2492,108,"Snohomish, Washington",26,0,"Snohomish, Washington, 5/2/2020"
2,5/3/2020,Snohomish,Washington,53061,2737,108,"Snohomish, Washington",245,0,"Snohomish, Washington, 5/3/2020"
3,5/4/2020,Snohomish,Washington,53061,2784,110,"Snohomish, Washington",47,2,"Snohomish, Washington, 5/4/2020"
4,5/5/2020,Snohomish,Washington,53061,2807,110,"Snohomish, Washington",23,0,"Snohomish, Washington, 5/5/2020"


## Comparing Counties Present in Datasets 

* This below code tells us which counties are present in the AQData but not in the covidData (because covidData is the more comprehensive dataset). We convert the counties in both dataframes to a set, that may be subtracted easily. 

In [8]:
covidMergeSet = covidData['county'].to_list()
AQMergeSet = AQData['County_Name'].to_list()

In [9]:
AQMergeSet = set(AQMergeSet)
covidMergeSet = set(covidMergeSet)

In [10]:
differenceSet = AQMergeSet - covidMergeSet

In [11]:
print('List of counties in AQData but not in covidData: \n\n\n', differenceSet)

List of counties in AQData but not in covidData: 


 {'Hampton City', 'Saint Charles', 'Suffolk City', 'Sainte Genevieve', 'Saint Clair', 'Saint Louis', 'Bronx', 'Queens', 'Fairbanks North Star ', 'Baltimore (City)', 'New York', 'Denali ', 'St. Louis City'}


## Cleaning up Discrepant Values

In [12]:
# Sainte Genevieve (AQData county) is listed as Ste. Genevieve in covidData, this code changes that. The code changes
#both the county name and uniqueID columns

covidData.loc[covidData.county == "Ste. Genevieve", "uniqueID"] = "Sainte Genevieve, Missouri"
covidData.loc[covidData.county == "Ste. Genevieve", "county"] = "Sainte Genevieve"

In [13]:
#Hampton City (AQData county) is a city in Washington county, New York. There is no data for Washington, New York. 
#We will change this county name to Washington, New York.  

AQData.loc[AQData.County_Name == 'Hampton City', 'uniqueID'] = 'Washington, New York'
AQData.loc[AQData.County_Name == 'Hampton City', 'County_Name'] = 'Washington'

In [14]:
### this is to ignore an error we get on the next line of code
pd.options.mode.chained_assignment = None  # default='warn'

In [15]:
# New York has several burrows listed (AQData county), but only New York City, New York is listed on the covidData. 
# We will clone New York (have the same data four times), and change the county names to Bronx, Queens,
#Richmond (equivalent to Staten Island), and New York.
#This would allow it to merge with the AQData. 

bronxData = covidData[covidData['county'] == 'New York City']
queensData = covidData[covidData['county'] == 'New York City']
richmondNYData = covidData[covidData['county'] == 'New York City']

bronxData['uniqueID'] = 'Bronx, New York'
bronxData['county'] = 'Bronx'

queensData['uniqueID'] = 'Queens, New York'
queensData['county'] = 'Queens'

richmondNYData['uniqueID'] = 'Richmond, New York'
richmondNYData['county'] = 'Richmond'

covidData.loc[covidData.county == 'New York City', 'county'] = 'New York'

appendList = [covidData, bronxData, queensData, richmondNYData]
covidData = pd.concat(appendList)

In [16]:
# Baltimore City (AQData county) is a city within baltimore county, which we already have data for. For this reason,
#data from baltimore city will be dropped 
AQData = AQData[AQData['County_Name'] != 'Baltimore (City)']

In [17]:
#Fairbanks North Star Borough  (covidData) was listed as Fairbanks North Star in AQdata. We will change AQData to match
AQData.loc[AQData.County_Name == 'Fairbanks North Star ', 'uniqueID'] = 'Fairbanks North Star Borough, Alaska'
AQData.loc[AQData.County_Name == 'Fairbanks North Star ', 'County_Name'] = 'Fairbanks North Star Borough'


In [18]:
#Changes St. Charles (covidData) to Saint Charles to match AQData
covidData.loc[covidData.county == "St. Charles", "uniqueID"] = "Saint Charles, Missouri"
covidData.loc[covidData.county == "St. Charles", "county"] = "Saint Charles"

In [19]:
# In covidData, St. Louis needs to be 'Saint Louis'. There is also duplicated
#regions here, as St. Louis City and St. Louis County are the same area, so we will drop St Louis City from AQdata. 

covidData.loc[covidData.county == "St. Louis", "uniqueID"] = "Saint Louis, Missouri"
covidData.loc[covidData.county == "St. Louis", "county"] = "Saint Louis"
AQData = AQData[AQData['County_Name'] != 'St. Louis City']

In [20]:
# Suffolk city (covidData) changed to Suffolk City to match with AQData
covidData.loc[(covidData['county'] == "Suffolk city"), "uniqueID"] = "Suffolk City, Virginia"
covidData.loc[(covidData['county'] == "Suffolk city"), "county"] = "Suffolk City"

In [21]:
# St. Clair (covidData) changed to Saint Claire to match with AQData
covidData.loc[(covidData['county'] == 'St. Clair') & (covidData['state'] == 'Illinois'), "uniqueID"] = "Saint Clair, Illinois"
covidData.loc[(covidData['county'] == 'St. Clair') & (covidData['state'] == 'Illinois'), "county"] = "Saint Clair"

In [22]:
# Denali Borough changed to Denali to match with AQData
covidData.loc[covidData.county == "Denali Borough", "uniqueID"] = "Denali, Alaska"
covidData.loc[covidData.county == "Denali Borough", "county"] = "Denali "

In [23]:
# Fairbanks North Star Borough changed to Fairbanks North Star to match with AQData
covidData.loc[covidData.county == "Fairbanks North Star Borough", "uniqueID"] = "Fairbanks North Star, Alaska"
covidData.loc[covidData.county == "Fairbanks North Star Borough", "county"] = "Fairbanks North Star "

In [24]:
# Charles City (covidData) changed to Charles to match with AQData
covidData.loc[covidData.county == "Charles City", "uniqueID"] = "Charles, Virginia"
covidData.loc[covidData.county == "Charles City", "county"] = "Charles"

In [25]:
### checking success of cleaning
covidMergeSet = covidData['county'].to_list()
AQMergeSet = AQData['County_Name'].to_list()
AQMergeSet = set(AQMergeSet)
covidMergeSet = set(covidMergeSet)
differenceSet = AQMergeSet - covidMergeSet
print('List of counties in AQData but not in covidData: \n\n\n', differenceSet)

List of counties in AQData but not in covidData: 


 {'Fairbanks North Star Borough'}


* This shows us that all of our counties in the AQdata can also be found in the covidData dataset. This means our merge should be successful. 

## Comparing mergeIDs Present in Datasets

In [26]:
# resetting merge index following dataset changes

covidData['mergeID'] =  covidData['county'] + ', ' + covidData['state'] + ', ' + covidData['date']
AQData['mergeId'] = AQData['County_Name'] + ', ' + AQData['State_Name'] + ', ' + AQData['Date_Local']


## check which merge IDs exist in AQData, but not in covidData
covidMergeSet = covidData['mergeID'].to_list()
AQMergeSet = AQData['mergeID'].to_list()
AQMergeSet = set(AQMergeSet)
covidMergeSet = set(covidMergeSet)
differenceSet = AQMergeSet - covidMergeSet
print('List of mergeIDs in AQData but not in covidData: \n\n\n', differenceSet)
print('Number of rows in AQdata, but not in covidData: ', len(differenceSet))

List of mergeIDs in AQData but not in covidData: 


 {'Custer, South Dakota, 5/20/2020', 'Hampton City, Virginia, 6/13/2020', 'Custer, South Dakota, 5/13/2020', 'Custer, South Dakota, 5/30/2020', 'Rosebud, Montana, 5/13/2020', 'Hampton City, Virginia, 6/21/2020', 'Avery, North Carolina, 5/2/2020', 'Custer, South Dakota, 5/16/2020', 'Butte, Idaho, 5/17/2020', 'Butte, Idaho, 8/8/2020', 'Butte, Idaho, 6/3/2020', 'Rosebud, Montana, 5/9/2020', 'Hampton City, Virginia, 6/4/2020', 'Phillips, Montana, 6/15/2020', 'Taylor, Wisconsin, 5/15/2020', 'Phillips, Montana, 5/8/2020', 'Denali , Alaska, 5/29/2020', 'Fergus, Montana, 5/17/2020', 'Powder River, Montana, 6/8/2020', 'Bell, Kentucky, 5/14/2020', 'Hampton City, Virginia, 6/22/2020', 'Fergus, Montana, 5/8/2020', 'Phillips, Montana, 5/6/2020', 'Phillips, Montana, 5/14/2020', 'Butte, Idaho, 5/21/2020', 'Butte, Idaho, 5/19/2020', 'Bell, Kentucky, 5/1/2020', 'Butte, Idaho, 5/30/2020', 'Phillips, Montana, 5/21/2020', 'Billings, North Dakota, 6/19/20

In [27]:
#providing new datasets after first round merge cleaning (1RMC)

AQData.to_csv('AQData1RMC.CSV')
covidData.to_csv('covidData1RMC.csv')

In [28]:
from collections import Counter

issueList = []
for x in differenceSet:
    cityState = ",".join(x.split(",", 2)[:2])
    issueList.append(cityState)
Counter(issueList)

Counter({'Custer, South Dakota': 33,
         'Hampton City, Virginia': 61,
         'Rosebud, Montana': 29,
         'Avery, North Carolina': 18,
         'Butte, Idaho': 102,
         'Phillips, Montana': 92,
         'Taylor, Wisconsin': 20,
         'Denali , Alaska': 55,
         'Fergus, Montana': 48,
         'Powder River, Montana': 68,
         'Bell, Kentucky': 17,
         'Billings, North Dakota': 50,
         'Palo Alto, Iowa': 7,
         'Jackson, South Dakota': 23,
         'Trego, Kansas': 21,
         'Monroe, Missouri': 26,
         'Weston, Wyoming': 24,
         'Mercer, North Dakota': 4,
         'Yancey, North Carolina': 2,
         'Lake, Minnesota': 6,
         'Forest, Wisconsin': 5})

## Research into states with issues 
* Custer, South Dakota-
* Hampton City, Virginia-
* Rosebud, Montana-
* Avery, North Carolina-
* Butte, Idaho-
* Phillips, Montana-
* Taylor, Wisconsin-
* Denali , Alaska-
* Fergus, Montana-
* Powder River, Montana-
* Bell, Kentucky-
* Billings, North Dakota-
* Palo Alto, Iowa-
* Jackson, South Dakota-
* Trego, Kansas-
* Monroe, Missouri-
* Weston, Wyoming-
* Mercer, North Dakota-
* Yancey, North Carolina-
* Lake, Minnesota-
* Forest, Wisconsin-